In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
data_pollution = pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", encoding = 'utf-8')

In [ ]:
base_geo_pour_merge = pd.read_csv("https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv", encoding = 'utf-8')
#base_geo_pour_merge_2 = base_geo_pour_merge['DEP'] and base_geo_pour_merge['REG']
base_geo_pour_merge

In [ ]:
base_geo_pour_merge.drop("DEP", axis = 1, inplace = True)
base_geo_pour_merge.drop("REG", axis = 1, inplace = True)

In [ ]:
data_pollution['DEP'] = data_pollution['INSEE commune'].str[:2]

In [ ]:
correspondance_reg = {
    84 : {1, 3, 7, 15, 26, 38, 42, 43, 63, 69, 73, 74}, #Auvergne-Rhône-Alpes
    27 : {21, 25, 39, 58, 70, 71, 89, 90}, #Bourgogne-Franche-Comté
    53 : {22, 29, 35, 56}, #Bretagne
    28 : {18, 28, 36, 37, 41, 45}, #Centre-Val de Loire
    94 : {'2A', '2B'}, #Corse
    44 : {8, 10, 51, 52, 54, 55, 57, 67, 68, 88}, #Grand Est
    32 : {2, 59, 60, 62, 80}, #Hauts-de-France
    11 : {75, 77, 78, 91, 92, 93, 94, 95}, #Île-de-France
    76 : {14, 27, 50, 61, 76}, #Normandie
    75 : {16, 17, 19, 23, 24, 33, 40, 47, 64, 79, 86, 87}, #Nouvelle-Aquitaine
    24 : {9, 11, 12, 30, 31, 32, 34, 46, 48, 65, 66, 81, 82}, #Occitanie
    52 : {44, 49, 53, 72, 85}, #Pays de la Loire
    93 : {4, 5, 6, 13, 83, 84} #Provence-Alpes-Côte d'Azur
}

In [ ]:
data_pollution['REG'] = data_pollution['DEP'].map(correspondance_reg)

In [ ]:
base_gen_pollution = pd.merge(data_pollution, base_geo_pour_merge, how='inner', 
                              left_on='INSEE commune', right_on='COM')
base_gen_pollution

In [ ]:
data_pollution.head()

In [ ]:
merging_list = data_pollution.columns

In [ ]:
merging_list = list(merging_list)

In [ ]:
print(merging_list)

In [ ]:
base_gen_pollution = base_gen_pollution[merging_list]

In [ ]:
base_gen_pollution.head()

In [ ]:
base_gen_pollution['REG'].unique()

In [ ]:
base_gen_pollution.shape

In [ ]:
len(base_gen_pollution['INSEE commune'].unique())

In [ ]:
len(base_gen_pollution['REG'].unique())

In [ ]:
len(base_gen_pollution['DEP'].unique())

In [ ]:
#probleme pour plus tard
#differences avec le nombre de regions et de departement dans l autre table 
# Gabrielle veut bien s'en charger

In [ ]:
# Filtrer les lignes pour lesquelles le département ou la région ne sont pas renseignés
#(ou au sens "au moins l'un des deux)
REG_DEP_na = data_pollution[(data_pollution['DEP'].isna())] #| (base_gen_pollution['REG'].isna())
REG_DEP_na.shape
#on perd les données de 1339 communes donc il faudrait trouver un moyen de prendre le département à partir du code commune

In [ ]:
tentative_groupes_1 = base_gen_pollution.drop(['Commune'], axis = 1).groupby('DEP').sum()

In [ ]:
tentative_groupes_1

In [ ]:
tentative_groupes_1 = tentative_groupes_1.drop(['INSEE commune', 'REG'], axis = 1)

In [ ]:
tentative_groupes_1

In [ ]:
tentative_groupes_1.to_csv('groupe_dep.csv')

In [ ]:
set_dep_littoral = (59, 62, 80, 76, 27, 14, 50, 35, 
                      22, 29, 56, 44, 85, 17, 33, 40, 
                    64, 66, 11, 34,
                    30, 13, 83, 0o6, '2A', '2B')

In [ ]:
print(set_dep_littoral)

In [ ]:
base_gen_pollution['DEP'].unique()

In [ ]:
#on a pas les DROM donc complique par rapport a l'autre base 

In [ ]:
# Importation de la base en local pour faire tourner le code main
base_gen_pollution.to_csv('base_pollution.csv', sep=',', index=False, encoding='utf-8')